In [1]:
import sys, os, itertools, sklearn
sys.path.append('/home/roquero/CausalAggregation/Code')
import pandas as pd
import numpy as np
import _pickle as pkl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import linear_model
from sklearn.linear_model import LassoCV, Lasso
from sklearn import datasets
from sklearn.metrics import auc
from scipy.interpolate import UnivariateSpline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%reload_ext autoreload
%autoreload 2
np.set_printoptions(suppress=True, edgeitems=10)
np.core.arrayprint._line_width = 80

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
from backfitting import Backfitting
from boosting import Boosting
from base_environment import BaseEnvironment
from collection_environment import CollectionEnvironment
from regression_method import PolynomialRegression, RandomForestRegression, DecisionTreeRegression
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures


/home/roquero/miniconda3/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


## Correct specification

In [2]:
ik_h, se_h = [], (lambda input_samples:input_samples)
ik_x1, se_x1 = ['h'], (lambda input_samples: 2*input_samples[0] + input_samples[1])
ik_x2, se_x2 = ['h', 'x1'], (lambda input_samples: input_samples[0] + input_samples[1] + input_samples[2])
ik_x3, se_x3 = ['x1', 'x2'], (lambda input_samples: -input_samples[0] * 2*input_samples[1] + input_samples[2])
ik_x4, se_x4 = ['x1', 'x3'], (lambda input_samples: np.log(1+np.abs(input_samples[0])) + input_samples[1] + input_samples[2])

ik_y, se_y = ['h','x1','x2','x3','x4'], (lambda input_samples:
                                        2*input_samples[0] 
                                         + (input_samples[1]>0)*1.
                                        + (input_samples[2]>0)*1.
                                         - (input_samples[2]>0)*(input_samples[3]<-1)*5.
                                         + (input_samples[1]<0)*(input_samples[4]<-1)*2.
                                         + (input_samples[1]<0)*(input_samples[2]<1)*(input_samples[3]<-1)*3.
                                         + 0.5*input_samples[5])
ik_x5, se_x5 = ['x2', 'x4', 'y'], (lambda input_samples: 2*input_samples[0] + input_samples[1] - input_samples[2] + input_samples[3])

fh = {'input_keys':ik_h, 'structural_eq': se_h}
f1 = {'input_keys':ik_x1, 'structural_eq': se_x1}
f2 = {'input_keys':ik_x2, 'structural_eq': se_x2}
f3 = {'input_keys':ik_x3, 'structural_eq': se_x3}
f4 = {'input_keys':ik_x4, 'structural_eq': se_x4}
f5 = {'input_keys':ik_x5, 'structural_eq': se_x5}
fy = {'input_keys':ik_y, 'structural_eq': se_y}

sh = lambda n_samples: np.random.normal(size=n_samples)
s1 = lambda n_samples: np.random.normal(size=n_samples)
s2 = lambda n_samples: np.random.normal(size=n_samples)
s3 = lambda n_samples: np.random.normal(size=n_samples)
s4 = lambda n_samples: np.random.normal(size=n_samples)
s5 = lambda n_samples: np.random.normal(size=n_samples)
sy = lambda n_samples: np.random.normal(size=n_samples)

topo_order = ['h','x1','x2','x3','x4','y','x5']
y_key = 'y'
x_key = ['x1','x2','x3','x4','x5']

structural_equation_dict = {'x1':f1, 'x2':f2, 'x3':f3, 'y':fy, 'x4':f4, 'x5':f5, 'h':fh}
disturbance_sampler_dict = {'x1':s1, 'x2':s2, 'x3':s3, 'y':sy, 'x4':s4, 'x5':s5, 'h':sh}

base = BaseEnvironment(structural_equation_dict, disturbance_sampler_dict, topo_order, y_key, x_key)
coll_env = CollectionEnvironment(base)

In [3]:
n_samples_list = [100,200,500,1000,2000,5000,10000]
loss_mean_list, loss_std_list, naive_loss_mean_list, naive_loss_std_list = [], [], [], []

for n_samples in n_samples_list:
    loss_list=[]
    naive_loss_list=[]
    n_rep=10
    for i in np.arange(n_rep):
        coll_env.reset_env()
        coll_env.add_env('e1', {'x1':{'type':'independent'},'x2':{'type':'independent'},'x3':{'type':'independent'}}, n_samples)
        coll_env.add_env('e2', {'x1':{'type':'independent'},'x4':{'type':'independent'}}, n_samples)
        coll_env.add_env('e3', {'x2':{'type':'independent'},'x3':{'type':'independent'}}, n_samples)
        
        backfit = Backfitting(DecisionTreeRegression, 
                              'boosting',
                              max_n_iter=10,
                              gap_convergence=1e-5,
                              warm_start=False, 
                              params_method={},
                              reweighting_candidates=True,
                              update_within_loop=False
                             )
        try:
            loss_list.append(backfit.fitCV(coll_env))
            naive_loss_list.append(backfit.fit_naiveCV(coll_env))   
        except:
            pass

    loss_mean_list.append(np.mean(np.stack(loss_list)[:,1]))
    loss_std_list.append(np.std(np.stack(loss_list)[:,1]))
    naive_loss_mean_list.append(np.mean(np.stack(naive_loss_list)[:,1]))
    naive_loss_std_list.append(np.std(np.stack(naive_loss_list)[:,1]))
print(loss_mean_list, loss_std_list, naive_loss_mean_list, naive_loss_std_list)

[8.508592401324002, 2.7585330478969086, 1.922993710961037, 0.8781046988675651, 0.4672995301650899, 0.29369820696375204, 0.11086227171898078] [5.059902264489634, 1.043297179731412, 0.9326258298903386, 0.7742364009132131, 0.21117211747667244, 0.2562535358991834, 0.03417496475616383] [4.83477303507988, 4.194141744853142, 4.141058283544043, 3.9898047169430924, 3.783363897570649, 3.570072075578609, 3.4623146910693867] [1.317486196656127, 0.46781936851116523, 0.36145167916514004, 0.16371896819577872, 0.11588000286035409, 0.07773490518376432, 0.0460229475396549]


In [ ]:
n_samples_list = [100,200,500,1000,2000,5000,10000]
loss_mean_list, loss_std_list, naive_loss_mean_list, naive_loss_std_list = [], [], [], []

for n_samples in n_samples_list:
    loss_list=[]
    naive_loss_list=[]
    n_rep=10
    for i in np.arange(n_rep):
        coll_env.reset_env()
        coll_env.add_env('e1', {'x1':{'type':'independent'}}, n_samples)
        coll_env.add_env('e2', {'x1':{'type':'independent'}}, n_samples)
        coll_env.add_env('e3', {'x2':{'type':'independent'}}, n_samples)
        
        backfit = Backfitting(DecisionTreeRegression, 
                              'boosting',
                              max_n_iter=10,
                              gap_convergence=1e-5,
                              warm_start=False, 
                              params_method={},
                              reweighting_candidates=True,
                              update_within_loop=False
                             )
        try:
            loss_list.append(backfit.fitCV(coll_env))
            naive_loss_list.append(backfit.fit_naiveCV(coll_env))   
        except:
            pass

    loss_mean_list.append(np.mean(np.stack(loss_list)[:,1]))
    loss_std_list.append(np.std(np.stack(loss_list)[:,1]))
    naive_loss_mean_list.append(np.mean(np.stack(naive_loss_list)[:,1]))
    naive_loss_std_list.append(np.std(np.stack(naive_loss_list)[:,1]))
print(loss_mean_list, loss_std_list, naive_loss_mean_list, naive_loss_std_list)

## Misspecified

In [ ]:
ik_h, se_h = [], (lambda input_samples:input_samples)
ik_x1, se_x1 = ['h'], (lambda input_samples: 2*input_samples[0] + input_samples[1])
ik_x2, se_x2 = ['h', 'x1'], (lambda input_samples: input_samples[0] + input_samples[1] + input_samples[2])
ik_x3, se_x3 = ['x1', 'x2'], (lambda input_samples: -input_samples[0] * 2*input_samples[1] + input_samples[2])
ik_x4, se_x4 = ['x1', 'x3'], (lambda input_samples: np.log(1+np.abs(input_samples[0])) + input_samples[1] + input_samples[2])

ik_y, se_y = ['h','x1','x2','x3','x4'], (lambda input_samples:
                                        2*input_samples[0] 
                                         + input_samples[1]
                                        - input_samples[2]
                                         - (input_samples[2]>0)*(input_samples[3]<-1)*5.
                                         + (input_samples[1]<0)*(input_samples[4]<-1)*2.
                                         + (input_samples[1]<0)*(input_samples[2]<1)*(input_samples[3]<-1)*3.
                                         + 0.5*input_samples[5])
ik_x5, se_x5 = ['x2', 'x4', 'y'], (lambda input_samples: 2*input_samples[0] + input_samples[1] - input_samples[2] + input_samples[3])

fh = {'input_keys':ik_h, 'structural_eq': se_h}
f1 = {'input_keys':ik_x1, 'structural_eq': se_x1}
f2 = {'input_keys':ik_x2, 'structural_eq': se_x2}
f3 = {'input_keys':ik_x3, 'structural_eq': se_x3}
f4 = {'input_keys':ik_x4, 'structural_eq': se_x4}
f5 = {'input_keys':ik_x5, 'structural_eq': se_x5}
fy = {'input_keys':ik_y, 'structural_eq': se_y}

sh = lambda n_samples: np.random.normal(size=n_samples)
s1 = lambda n_samples: np.random.normal(size=n_samples)
s2 = lambda n_samples: np.random.normal(size=n_samples)
s3 = lambda n_samples: np.random.normal(size=n_samples)
s4 = lambda n_samples: np.random.normal(size=n_samples)
s5 = lambda n_samples: np.random.normal(size=n_samples)
sy = lambda n_samples: np.random.normal(size=n_samples)

topo_order = ['h','x1','x2','x3','x4','y','x5']
y_key = 'y'
x_key = ['x1','x2','x3','x4','x5']

structural_equation_dict = {'x1':f1, 'x2':f2, 'x3':f3, 'y':fy, 'x4':f4, 'x5':f5, 'h':fh}
disturbance_sampler_dict = {'x1':s1, 'x2':s2, 'x3':s3, 'y':sy, 'x4':s4, 'x5':s5, 'h':sh}

base = BaseEnvironment(structural_equation_dict, disturbance_sampler_dict, topo_order, y_key, x_key)
coll_env = CollectionEnvironment(base)

In [ ]:
n_samples_list = [100,200,500,1000,2000,5000,10000]
loss_mean_list, loss_std_list, naive_loss_mean_list, naive_loss_std_list = [], [], [], []

for n_samples in n_samples_list:
    loss_list=[]
    naive_loss_list=[]
    n_rep=10
    for i in np.arange(n_rep):
        coll_env.reset_env()
        coll_env.add_env('e1', {'x1':{'type':'independent'},'x2':{'type':'independent'},'x3':{'type':'independent'}}, n_samples)
        coll_env.add_env('e2', {'x1':{'type':'independent'},'x4':{'type':'independent'}}, n_samples)
        coll_env.add_env('e3', {'x2':{'type':'independent'},'x3':{'type':'independent'}}, n_samples)
        
        backfit = Backfitting(DecisionTreeRegression, 
                              'boosting',
                              max_n_iter=10,
                              gap_convergence=1e-5,
                              warm_start=False, 
                              params_method={},
                              reweighting_candidates=True,
                              update_within_loop=False
                             )
        try:
            loss_list.append(backfit.fitCV(coll_env))
            naive_loss_list.append(backfit.fit_naiveCV(coll_env))   
        except:
            pass

    loss_mean_list.append(np.mean(np.stack(loss_list)[:,1]))
    loss_std_list.append(np.std(np.stack(loss_list)[:,1]))
    naive_loss_mean_list.append(np.mean(np.stack(naive_loss_list)[:,1]))
    naive_loss_std_list.append(np.std(np.stack(naive_loss_list)[:,1]))
print(loss_mean_list, loss_std_list, naive_loss_mean_list, naive_loss_std_list)

## Misspecified

In [ ]:
ik_h, se_h = [], (lambda input_samples:input_samples)
ik_x1, se_x1 = ['h'], (lambda input_samples: 2*input_samples[0] + input_samples[1])
ik_x2, se_x2 = ['h', 'x1'], (lambda input_samples: input_samples[0] + input_samples[1] + input_samples[2])
ik_x3, se_x3 = ['x1', 'x2'], (lambda input_samples: -input_samples[0] * 2*input_samples[1] + input_samples[2])
ik_x4, se_x4 = ['x1', 'x3'], (lambda input_samples: np.log(1+np.abs(input_samples[0])) + input_samples[1] + input_samples[2])

ik_y, se_y = ['h','x1','x2','x3','x4'], (lambda input_samples:
                                         0.3*input_samples[0] 
                                         + 1*input_samples[1]**2
                                         #- 0.5*input_samples[3]*input_samples[4]
                                         - 1*input_samples[2]*input_samples[4] + 0.3*input_samples[5])
ik_x5, se_x5 = ['x2', 'x4', 'y'], (lambda input_samples: 2*input_samples[0] + input_samples[1] - input_samples[2] + input_samples[3])

fh = {'input_keys':ik_h, 'structural_eq': se_h}
f1 = {'input_keys':ik_x1, 'structural_eq': se_x1}
f2 = {'input_keys':ik_x2, 'structural_eq': se_x2}
f3 = {'input_keys':ik_x3, 'structural_eq': se_x3}
f4 = {'input_keys':ik_x4, 'structural_eq': se_x4}
f5 = {'input_keys':ik_x5, 'structural_eq': se_x5}
fy = {'input_keys':ik_y, 'structural_eq': se_y}

sh = lambda n_samples: np.random.normal(size=n_samples)
s1 = lambda n_samples: np.random.normal(size=n_samples)
s2 = lambda n_samples: np.random.normal(size=n_samples)
s3 = lambda n_samples: np.random.normal(size=n_samples)
s4 = lambda n_samples: np.random.normal(size=n_samples)
s5 = lambda n_samples: np.random.normal(size=n_samples)
sy = lambda n_samples: np.random.normal(size=n_samples)

topo_order = ['h','x1','x2','x3','x4','y','x5']
y_key = 'y'
x_key = ['x1','x2','x3','x4','x5']

structural_equation_dict = {'x1':f1, 'x2':f2, 'x3':f3, 'y':fy, 'x4':f4, 'x5':f5, 'h':fh}
disturbance_sampler_dict = {'x1':s1, 'x2':s2, 'x3':s3, 'y':sy, 'x4':s4, 'x5':s5, 'h':sh}

base = BaseEnvironment(structural_equation_dict, disturbance_sampler_dict, topo_order, y_key, x_key)
coll_env = CollectionEnvironment(base)

In [ ]:
n_samples_list = [100,200,500,1000,5000,10000,50000,100000]
loss_mean_list, loss_std_list, naive_loss_mean_list, naive_loss_std_list = [], [], [], []

for n_samples in n_samples_list:
    loss_list=[]
    naive_loss_list=[]
    n_rep=10
    for i in np.arange(n_rep):
        coll_env.reset_env()
        coll_env.add_env('e1', {'x1':{'type':'independent'},'x2':{'type':'independent'},'x3':{'type':'independent'}}, n_samples)
        coll_env.add_env('e2', {'x1':{'type':'independent'},'x4':{'type':'independent'}}, n_samples)
        coll_env.add_env('e3', {'x2':{'type':'independent'},'x3':{'type':'independent'}}, n_samples)
        
        backfit = Backfitting(DecisionTreeRegression, 
                              'boosting',
                              max_n_iter=10,
                              gap_convergence=1e-5,
                              warm_start=False, 
                              params_method={},
                              reweighting_candidates=True,
                              update_within_loop=False
                             )
        try:
            loss_list.append(backfit.fitCV(coll_env, range_min=-3, range_max=2, range_step =0.2, print_selected=True))
            naive_loss_list.append(backfit.fit_naiveCV(coll_env))   
        except:
            pass

    loss_mean_list.append(np.mean(np.stack(loss_list)[:,1]))
    loss_std_list.append(np.std(np.stack(loss_list)[:,1]))
    naive_loss_mean_list.append(np.mean(np.stack(naive_loss_list)[:,1]))
    naive_loss_std_list.append(np.std(np.stack(naive_loss_list)[:,1]))
print(loss_mean_list, loss_std_list, naive_loss_mean_list, naive_loss_std_list)

In [ ]:
print(loss_mean_list, loss_std_list, naive_loss_mean_list, naive_loss_std_list)